# Download GPKG

## Grid

In [ ]:
import os
import requests

DATA_PATH = "C:/Users/tiago/Documents/Workspace/UFG/pfc-cc-2025/data"
os.makedirs(DATA_PATH, exist_ok=True)

url = "https://zenodo.org/records/15631655/files/gpw_grassland_fscs.vi.vhr_grid.samples_20000101_20241231_go_epsg.4326_v2.gpkg?download=1"

filename = os.path.join(DATA_PATH, "gpw_grassland_fscs.vi.vhr_grid.samples_20000101_20241231_go_epsg.4326_v2.gpkg")

response = requests.get(url, stream=True)
response.raise_for_status()

with open(filename, "wb") as f:
    for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)

print(f"Download concluído: {filename}")

## Tile

In [ ]:
import os
import requests

DATA_PATH = "C:/Users/tiago/Documents/Workspace/UFG/pfc-cc-2025/data"
os.makedirs(DATA_PATH, exist_ok=True)

url = "https://zenodo.org/records/15631655/files/gpw_grassland_fscs.vi.vhr_tile.samples_20000101_20241231_go_epsg.4326_v1.gpkg?download=1"

filename = os.path.join(DATA_PATH, "gpw_grassland_samples.gpkg")

response = requests.get(url, stream=True)
response.raise_for_status()

with open(filename, "wb") as f:
    for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)

print(f"Download concluído: {filename}")

# Retreive AlphaEarth Images

In [2]:
import ee

ee.Authenticate()
ee.Initialize(project="ee-tiagogoncalves")

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


In [3]:
import os
import geopandas as gpd

DATA_PATH = "C:/Users/tiago/Documents/Workspace/UFG/pfc-cc-2025/data"
FEATURE_PATH = "C:/Users/tiago/Documents/Workspace/UFG/pfc-cc-2025/data/gee_images"

os.makedirs(FEATURE_PATH, exist_ok=True)

In [ ]:
def get_download_url(geometry, year):
    left_bottom = [geometry[0], geometry[1]]
    right_top = [geometry[2], geometry[3]]

    geometry = ee.Geometry.Rectangle([left_bottom, right_top])

    s_date = ee.Date.fromYMD(year, 1, 1)
    e_date = s_date.advance(1, 'year')
                
    collection = ee.ImageCollection("GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL")
    
    image = collection.filterBounds(geometry).filterDate(s_date, e_date).first()
    
    return image.getDownloadURL({
        'scale': 10,
        'region': geometry,
        'format': 'GEO_TIFF'
    })


def download_from_url(filename, url):
    import requests
    response = requests.get(url)

    output_filename = f"{filename}.tif"
    output_path = os.path.join(FEATURE_PATH, output_filename)
    
    if response.status_code == 200:
        with open(output_path, 'wb') as f:
            f.write(response.content)
        print(f"    ✓ Salvo: {filename}")
    else:
        print(f"    ❌ Erro ao baixar: Status {response.status_code}")

In [6]:
gpw_grassland_samples = gpd.read_file(os.path.join(DATA_PATH, "gpw_grassland_fscs.vi.vhr_tile.samples_20000101_20241231_go_epsg.4326_v1.gpkg"))

gpw_grassland_samples_espg_4326 = gpw_grassland_samples.to_crs("EPSG:4326")

In [10]:
from time import sleep

for idx, row in gpw_grassland_samples_espg_4326.iterrows():
    geometry = row.geometry.bounds

    for _ in range(2):
        try:
            print(f"Baixando imagem para tile_id: {row.vi_tile_id} (índice {idx})...")

            url = get_download_url(geometry, 2024)

            print(f"  URL obtida, iniciando download... {url}")

            download_from_url(row.vi_tile_id, url)

            break
        except Exception as e:
            print(f"    ❌ Erro ao baixar a imagem: {e}. Tentando novamente...")
            sleep(2)

print("✅ Processamento concluído!")

saved_files = [f for f in os.listdir(FEATURE_PATH) if f.endswith('.tif')]
print(f"Total de arquivos GeoTIFF salvos: {len(saved_files)}")

Baixando imagem para tile_id: 1-mvp (índice 0)...
    ❌ Erro ao baixar a imagem: Bands parameter must be a list.. Tentando novamente...
Baixando imagem para tile_id: 1-mvp (índice 0)...
    ❌ Erro ao baixar a imagem: Bands parameter must be a list.. Tentando novamente...


KeyboardInterrupt: 